In [1]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import plotly

project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.append(project_root)

from preprocessing.video_tracking import read_metrica_event_data

#Hiển thị toàn bộ thông tin cột dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
matchID = 2500045
df_events, df_formations = read_metrica_event_data(game=matchID, saved=True)

df_events.head(10)


____Xử lý dữ liệu trận: 2500045___


,ID,matchID,matchPeriod,eventSec,eventSecEnd,startFrame,endFrame,eventName,subEventName,teamID,Team,posBeforeXMeters,posBeforeYMeters,posAfterXMeters,posAfterYMeters,playerID,playerName,playerPosition,toPlayerID,toPlayerName,homeTeamID,awayTeamID,accurate,Goal,ownGoal
0,552130,2500045,1H,2.036874,3.236874,50,80,Pass,Simple pass,1611,Away,52.50,34.00,46.20,47.60,7934,J. Lingard,Midfielder,70122.0,N. Matić,1625,1611,1,0,0
1,552131,2500045,1H,7.181832,11.786454,179,294,Pass,Simple pass,1611,Away,46.20,47.60,29.40,60.52,70122,N. Matić,Midfielder,7938.0,A. Valencia,1625,1611,1,0,0
2,552132,2500045,1H,11.786454,12.986454,294,324,Pass,Simple pass,1611,Away,29.40,60.52,9.45,40.12,7938,A. Valencia,Defender,7910.0,David de Gea,1625,1611,1,0,0
3,552133,2500045,1H,20.541311,23.179631,513,579,Pass,Launch,1611,Away,9.45,40.12,71.40,51.00,7910,David de Gea,Goalkeeper,105339.0,Fernandinho,1625,1611,1,0,0
4,552134,2500045,1H,23.179631,23.227421,579,580,Duel,Ground loose ball duel,1625,Home,33.60,17.00,35.70,13.60,105339,Fernandinho,Midfielder,7905.0,R. Lukaku,1625,1611,0,0,0
5,552135,2500045,1H,23.227421,24.454220,580,611,Duel,Ground loose ball duel,1611,Away,71.40,51.00,69.30,54.40,7905,R. Lukaku,Forward,7905.0,R. Lukaku,1625,1611,1,0,0
6,552136,2500045,1H,24.454220,25.654220,611,641,Pass,Head pass,1611,Away,69.30,54.40,94.50,38.08,7905,R. Lukaku,Forward,71654.0,Ederson,1625,1611,0,0,0
7,552137,2500045,1H,31.726154,32.819092,793,820,Pass,Hand pass,1625,Home,10.50,29.92,8.40,42.84,71654,Ederson,Goalkeeper,8307.0,V. Kompany,1625,1611,1,0,0
8,552138,2500045,1H,32.819092,35.212859,820,880,Pass,Simple pass,1625,Home,8.40,42.84,8.40,23.80,8307,V. Kompany,Defender,70086.0,N. Otamendi,1625,1611,1,0,0
9,552139,2500045,1H,35.212859,38.491035,880,962,Pass,Simple pass,1625,Home,8.40,23.80,13.65,53.04,70086,N. Otamendi,Defender,8307.0,V. Kompany,1625,1611,1,0,0
